<a href="https://colab.research.google.com/github/paiml/python_for_datascience/blob/master/Lesson16_Python_For_Data_Science_CaseStudies.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lesson 16: Case Studies

## 16.4 Ludwig (Open Source AutoML)

**Github Project URL**:  https://uber.github.io/ludwig/

![alt text](https://user-images.githubusercontent.com/58792/52920000-f78d8c00-32bc-11e9-8e5e-adf53f1b8a37.png)

### Install Ludwig

In [0]:
!pip install --upgrade numpy #must restart colab runtime
!pip install --upgrade scikit-image
!pip install -q ludwig
!python -m spacy download en 

Requirement already up-to-date: numpy in /usr/local/lib/python3.6/dist-packages (1.16.1)
    100% |████████████████████████████████| 25.3MB 1.9MB/s 
    100% |████████████████████████████████| 2.0MB 18.3MB/s 
    100% |████████████████████████████████| 706kB 25.2MB/s 
featuretools 0.4.1 has requirement pandas>=0.23.0, but you'll have pandas 0.22.0 which is incompatible.
albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.8 which is incompatible.
  Found existing installation: Pillow 4.0.0
    Uninstalling Pillow-4.0.0:
      Successfully uninstalled Pillow-4.0.0
  Found existing installation: dask 0.20.2
    Uninstalling dask-0.20.2:
      Successfully uninstalled dask-0.20.2
  Found existing installation: scikit-image 0.13.1
    Uninstalling scikit-image-0.13.1:
      Successfully uninstalled scikit-image-0.13.1



    Linking successful
    /usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
    /usr/local/lib/python3.6/dist-packages/spacy/data/en

    You can now load the model via spacy.load('en')



### Basic Ideas

* **Training Models**
* **Prediction (Inference)**
* **Datatypes**
 - binary
 - numerical
 - category
 - set
 - bag
 - sequence
 - text
 - timeseries
 - image



### Topic Modeling Example

In [37]:
!wget https://raw.githubusercontent.com/uchidalab/book-dataset/master/Task1/book30-listing-train.csv
!wget https://raw.githubusercontent.com/noahgift/recommendations/master/model_definition.yaml

--2019-02-18 01:16:28--  https://raw.githubusercontent.com/uchidalab/book-dataset/master/Task1/book30-listing-train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9728786 (9.3M) [text/plain]
Saving to: ‘book30-listing-train.csv.1’

book30-listing-trai 100%[===================>]   9.28M  --.-KB/s    in 0.1s    

2019-02-18 01:16:29 (66.7 MB/s) - ‘book30-listing-train.csv.1’ saved [9728786/9728786]

--2019-02-18 01:16:30--  https://raw.githubusercontent.com/noahgift/recommendations/master/model_definition.yaml
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting respon

#### Ingest

In [1]:
import pandas as pd
df = pd.read_csv("https://media.githubusercontent.com/media/noahgift/recommendations/master/data/book30-listing-train-with-headers.csv")
df = df.drop("Unnamed: 0", axis=1)
df.head()

,ASIN,FILENAME,IMAGE URL,TITLE,AUTHOR,CATEGORYID,CATEGORY
0,1404803335,1404803335.jpg,http://ecx.images-amazon.com/images/I/51UJnL3T...,"Magnets: Pulling Together, Pushing Apart (Amaz...",Natalie M. Rosinsky,4,Children's Books
1,1446276082,1446276082.jpg,http://ecx.images-amazon.com/images/I/51MGUKhk...,Energy Security (SAGE Library of International...,NaN,10,Engineering & Transportation
2,1491522666,1491522666.jpg,http://ecx.images-amazon.com/images/I/51qKvjsi...,An Amish Gathering: Life in Lancaster County,Beth Wiseman,9,Christian Books & Bibles
3,970096410,0970096410.jpg,http://ecx.images-amazon.com/images/I/51qoUENb...,City of Rocks Idaho: A Climber's Guide (Region...,Dave Bingham,26,Sports & Outdoors
4,8436808053,8436808053.jpg,http://ecx.images-amazon.com/images/I/41aDW5pz...,"Como vencer el insomnio. Tecnicas, reglas y co...",Choliz Montanes,11,"Health, Fitness & Dieting"


In [0]:
df.to_csv("book30-listing-train-with-headers.csv")

#### EDA

**Columns**

In [35]:
df.columns

Index(['ASIN', 'FILENAME', 'IMAGE URL', 'TITLE', 'AUTHOR', 'CATEGORYID',
       'CATEGORY'],
      dtype='object')

**Shape**

In [36]:
df.shape

(51299, 7)

#### Training w/Ludwig

In [6]:
!head book30-listing-train-with-headers.csv

,ASIN,FILENAME,IMAGE URL,TITLE,AUTHOR,CATEGORYID,CATEGORY
0,1404803335,1404803335.jpg,http://ecx.images-amazon.com/images/I/51UJnL3Tx6L.jpg,"Magnets: Pulling Together, Pushing Apart (Amazing Science)",Natalie M. Rosinsky,4,Children's Books
1,1446276082,1446276082.jpg,http://ecx.images-amazon.com/images/I/51MGUKhkyhL.jpg,Energy Security (SAGE Library of International Security),,10,Engineering & Transportation
2,1491522666,1491522666.jpg,http://ecx.images-amazon.com/images/I/51qKvjsi3ML.jpg,An Amish Gathering: Life in Lancaster County,Beth Wiseman,9,Christian Books & Bibles
3,970096410,0970096410.jpg,http://ecx.images-amazon.com/images/I/51qoUENb1CL.jpg,City of Rocks Idaho: A Climber's Guide (Regional Rock Climbing Series),Dave Bingham,26,Sports & Outdoors
4,8436808053,8436808053.jpg,http://ecx.images-amazon.com/images/I/41aDW5pzZBL.jpg,"Como vencer el insomnio. Tecnicas, reglas y consejos practicos para dormir mejor (BIBLIOTECA PRACTICA) (Spanish Edition)",Choliz Montanes,11,"Health, Fi

In [5]:
!cat model_definition.yaml

input_features:
    -
        name: TITLE
        type: text
        encoder: parallel_cnn
        level: word

output_features:
    -
        name: CATEGORY
        type: category

In [3]:
!ludwig experiment --data_csv book30-listing-train-with-headers.csv --model_definition_file model_definition.yaml


 _         _        _      
| |_  _ __| |_ __ _(_)__ _ 
| | || / _` \ V  V / / _` |
|_|\_,_\__,_|\_/\_/|_\__, |
                     |___/ 
ludwig v0.1.0 - Experiment

Experiment name: experiment
Model name: run
Output path: results/experiment_run_0

ludwig_version: '0.1.0'
command: ('/usr/local/bin/ludwig experiment --data_csv '
 'book30-listing-train-with-headers.csv --model_definition_file '
 'model_definition.yaml')
dataset_type: 'book30-listing-train-with-headers.csv'
model_definition: {   'combiner': {'type': 'concat'},
    'input_features': [   {   'encoder': 'parallel_cnn',
                              'level': 'word',
                              'name': 'TITLE',
                              'tied_weights': None,
                              'type': 'text'}],
    'output_features': [   {   'dependencies': [],
                               'loss': {   'class_distance_temperature': 0,
                                           'class_weights': 1,
                           

## 16.5 Sklearn Algorithm Cheatsheet

## 16.6 Recommendations

### Install Surprise

In [0]:
 !pip install -q scikit-surprise

In [0]:
from surprise import SVD
from surprise import Dataset
from surprise.model_selection import cross_validate

# Load the movielens-100k dataset (download it if needed).
data = Dataset.load_builtin('ml-100k')

# Use the famous SVD algorithm.
algo = SVD()

# Run 5-fold cross-validation and print results.
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Dataset ml-100k could not be found. Do you want to download it? [Y/n] y
Trying to download dataset from http://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /root/.surprise_data/ml-100k
Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9362  0.9408  0.9329  0.9323  0.9394  0.9363  0.0034  
MAE (testset)     0.7384  0.7408  0.7371  0.7363  0.7375  0.7380  0.0016  
Fit time          5.97    6.01    6.17    6.00    5.99    6.03    0.07    
Test time         0.16    0.24    0.15    0.24    0.15    0.19    0.04    


{'fit_time': (5.973851203918457,
  6.005405426025391,
  6.169473171234131,
  6.002313852310181,
  5.9893479347229),
 'test_mae': array([0.73838687, 0.7408282 , 0.73707111, 0.73626816, 0.73751054]),
 'test_rmse': array([0.93621549, 0.94080079, 0.93292323, 0.93234383, 0.93936283]),
 'test_time': (0.15648674964904785,
  0.24050045013427734,
  0.1525723934173584,
  0.24495315551757812,
  0.15084218978881836)}